In [2]:
import requests 
from bs4 import BeautifulSoup
import pandas as pd
import re

In [7]:
URL = 'https://bitinfocharts.com/comparison/bitcoin-tweets.html#alltime'
response = requests.get(URL)
soup = BeautifulSoup(response.text, 'html.parser')
scripts = soup.find_all('script')

In [31]:
def parse(string_list):
    '''
    parse list of strings within the script tag
    [date, volume]
    '''
    clean = re.sub('[\[\],\s]', '', string_list)
    splitted = re.split("[\'\"]", clean)
    values_only = [s for s in splitted if s != '']
    return values_only

def process_scripts():
    '''
    Scrape URL script tag and extract tweet volume & respective date
    '''
    dates = []
    tweets = []
    
    for script in scripts:
        if 'd = new Dygraph(document.getElementById("container")' in script.text:
            str_lst = script.text
            str_lst = '[[' + str_lst.split('[[')[-1]
            str_lst = str_lst.split(']]')[0] +']]'
            str_lst = str_lst.replace('new Date(', '').replace(')', '')
            data = parse(str_lst)
    
    for each in data:
        if (data.index(each) % 2) == 0:
            dates.append(each)
        else:
            tweets.append(each)
            
    return dates, tweets

def create_dataframe():
    '''
    Create dataframe from scraped twitter volume and dates
    '''
    dates, tweets = process_scripts()
    df = pd.DataFrame(list(zip(dates, tweets)), columns=['Date', 'Tweet Volume'])
    return df

def export_data(df):
    '''
    Save data
    '''
    df.to_csv('../../data/Tweets/BTC_tweet_volume.csv')

In [32]:
df = create_dataframe()

In [33]:
df.head()

,Date,Tweet Volume
0,2014/04/09,8193
1,2014/04/10,15039
2,2014/04/11,14907
3,2014/04/12,7582
4,2014/04/13,10674


In [34]:
export_data(df)